In [ ]:
from collections import namedtuple
import pathlib
import random

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from orchid import (net_quantity as opq,
                    measurement as om,
                    project as op,
                    project_loader as opl,
                    reference_origins as origin,
                    unit_system as units)

In [ ]:
import toolz.curried as toolz

In [ ]:
from Orchid.FractureDiagnostics import WellReferenceFrameXy, DepthDatum
from Orchid.FractureDiagnostics.RatioTypes import ProppantConcentration, SlurryRate
import UnitsNet
from System import Array, Int32

In [ ]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

In [ ]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

In [ ]:
project_units = toolz.valmap(lambda p: p.ProjectUnits, projects)
project_units

In [ ]:
def get_project_wells(proj):
    return proj.Wells.Items

def collect_well_names(wells):
    return {w.Name: w for w in wells}

project_wells = toolz.pipe(
    projects,
    toolz.valmap(get_project_wells),
    toolz.valmap(collect_well_names),
)
project_wells

In [ ]:
bakken_wells = project_wells['bakken']

In [ ]:
bakken_demo_1h = toolz.get_in(['bakken', 'Demo_1H'], project_wells)

In [ ]:
trajectory = bakken_demo_1h.Trajectory

In [ ]:
eastings = trajectory.GetEastingArray(WellReferenceFrameXy.Project)

In [ ]:
len(eastings)

In [ ]:
toolz.pipe(eastings,
           toolz.take(5),
           toolz.map(str),
           list)

In [ ]:
np_eastings = toolz.pipe(
    eastings,
    toolz.map(opq.deprecated_as_measurement(units.UsOilfield.LENGTH)),
    toolz.map(lambda m: m.magnitude),
    list,
    np.array,
)

In [ ]:
measured_eastings = om.registry.Quantity(np_eastings, om.registry.ft)

In [ ]:
measured_eastings[:5]